In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta
from apps.environment.service_layer.data_transformer import DataTransformer
import pandas as pd
from joblib import load
import pathlib
import pytz

In [ ]:
starting_time_step = datetime(year=2016, month=1, day=1)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 900, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
dformer = DataTransformer(env_config=env_config)
dformer.retreive_stocks_buffers()
dformer.retreive_commodities_buffers()
dformer.retreive_cryptos_buffers()

In [ ]:
from copy import deepcopy


(current_file_path, new_file_path) = DataTransformer.get_new_file_path()

rf = load(pathlib.Path(current_file_path))

nwb_starting_time_step = datetime(year=2023, month=6, day=1)
now_time = pytz.utc.localize(datetime.now())

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 500, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 100,
    "no_of_cmmdts": 14,
    "is_live": True
} 
nwb_dformer = DataTransformer(env_config=nwb_env_config)
nwb_dfs = nwb_dformer.create_input_dataframes()
nwb_market = MarketSimulator(env_config=nwb_env_config, data_frames=nwb_dfs)
nwb_forest = Forester(market=nwb_market)
total_potfolio = nwb_market.wallet_state
latest_action = None
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
initial_total_potfolio = total_potfolio
print(total_potfolio)
print(nwb_market.the_current_time_step, now_time)
for indx in range(500):
    if indx < 7:
        acts = np.full(100, 15)
        rand_acts = nwb_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        flat_state = np.array([nwb_forest.get_flattened_states(nwb_market.state)])
        my_pred = rf.predict(flat_state)
        acts = np.full(100, 15)
        # acts = my_pred[0]
        some_acts = deepcopy(acts)
        print(nwb_market.the_current_time_step)
        acts_list = []
        iny = 1
        rand_acts = nwb_forest.act_random(acts, use_best_stks=False)
        for an_act in rand_acts:
            # if an_act <10:
            #     print(f"dddddhere dddddhere dddddhere dddddhere {iny}")
            acts_list.append((iny, an_act))
            iny=iny+1
        print(acts_list)
        # print(nwb_market.the_current_time_step.day, now_time.day)
        if nwb_market.the_current_time_step.day >= now_time.day and (nwb_market.the_current_time_step.month >= now_time.month-1):
            latest_action = acts
            break
        nwb_market.step(rand_acts)

# print(nwb_market.wallet_state)
# print(nwb_market.state)

In [ ]:
(a,b,c) = nwb_market.state
print(a[0].flatten())

In [ ]:
total_potfolio = nwb_market.wallet_state
for ind in range(100):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))
final_total_potfolio = total_potfolio
gain = final_total_potfolio - initial_total_potfolio
print("gain", gain)

In [ ]:
print(latest_action)